In [17]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [7]:
df = pd.read_csv('Pasta1.csv',sep=';')
df

,Marca,Serie,Tipo de processador,Memoria em GB,Modelo de placa de video
0,ACER,Acer Notebook Gamer Nitro 5,Intel Core i5,8,NVIDIA GeForce GTX 1650
1,Dell,Notebook Dell Inspiron,Intel Core i7,16,NVIDIA
2,VAIO,Vaio FE14,Core i7 2.7 GHz,8,Intel Iris Xe Graphics
3,Lenovo,Gaming 3,AMD R Series,8,NVIDIA GeForce GTX 1650


Será necessario re-organizar o dataframe para poder aplicar os métodos ordinais da pesquisa operacional

# Método SAPEVO-M

### Passo 1: Avaliação dos critérios

avaliação do decisor 1

In [34]:
# Matriz quadrada tres por tres
av_criterios = {
    "tipo de processador": pd.Series([0, -1, 0], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
    "Memoria em GB": pd.Series([1, 0, 2], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
    "Modelo de placa de video": pd.Series([0, -2, 0], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
}
df_criterios = pd.DataFrame(av_criterios)

Avaliação do primeiro decisor

In [35]:
df_criterios

,tipo de processador,Memoria em GB,Modelo de placa de video
tipo de processador,0,1,0
Memoria em GB,-1,0,-2
Modelo de placa de video,0,2,0


In [36]:
dt_n = df_criterios.values

In [37]:
dt_n

array([[ 0,  1,  0],
       [-1,  0, -2],
       [ 0,  2,  0]], dtype=int64)

In [40]:
ponto_criterio = np.sum(dt_n, axis=1)

In [41]:
ponto_criterio

array([ 1, -3,  2], dtype=int64)

normalização da pontuação,
fórmula matemática: v = aij - min aij / max aij - min aij

In [159]:
norm_criterios = (ponto_criterio[0] - ponto_criterio[1]) / (ponto_criterio[-1] - ponto_criterio[-2])


In [160]:
norm2_criterios = (ponto_criterio[1] - ponto_criterio[1]) / (ponto_criterio[-1] - ponto_criterio[-2])


In [161]:
norm3_criterios = (ponto_criterio[2] - ponto_criterio[1]) / (ponto_criterio[-1] - ponto_criterio[-2])


In [48]:
#array da normalização
pont_narl =np.array([norm_criterios,norm2_criterios,norm3_criterios])
pont_narl

array([0.8, 0. , 1. ])

segundo o metodo SAPEVO_M caso algum valor do vetor dos criterios for 0, como é o caso do segundo valor do vetor gerado pelo decisor 1, ele deve ser substituido por 1% do menor subsequente.

In [50]:
# pesos finais do decisor 1
pont_narl =np.array([norm_criterios,norm_criterios * 0.01,norm3_criterios])
pont_narl

array([0.8  , 0.008, 1.   ])

Avaliação do decisor 2

In [58]:
# Matriz quadrada tres por tres
av_criterios_dc2 = {
    "tipo de processador": pd.Series([0, -2, -1], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
    "Memoria em GB": pd.Series([2, 0, 0], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
    "Modelo de placa de video": pd.Series([1, 0, 0], index=["tipo de processador","Memoria em GB","Modelo de placa de video"]),
}
df_criterios_dc2 = pd.DataFrame(av_criterios_dc2)

In [59]:
df_criterios_dc2

,tipo de processador,Memoria em GB,Modelo de placa de video
tipo de processador,0,2,1
Memoria em GB,-2,0,0
Modelo de placa de video,-1,0,0


In [61]:
dt_n2 = df_criterios_dc2.values
dt_n2 

array([[ 0,  2,  1],
       [-2,  0,  0],
       [-1,  0,  0]], dtype=int64)

In [162]:
ponto_criterio2 = np.sum(dt_n2, axis=1)


In [163]:
norm_criterios_d2 = (ponto_criterio2[0] - ponto_criterio2[1]) / (ponto_criterio2[0] - ponto_criterio2[-2])


In [164]:
norm1_criterios_d2 = (ponto_criterio2[1]- ponto_criterio2[1]) / (ponto_criterio2[0] - ponto_criterio2[-2])


In [165]:
norm2_criterios_d2 = (ponto_criterio2[2]- ponto_criterio2[1]) / (ponto_criterio2[0] - ponto_criterio2[-2])


In [166]:
pont_narl_d2 =np.array([norm_criterios_d2,norm1_criterios_d2,norm2_criterios_d2])


In [168]:
#peso final
pont_narl_d2 =np.array([norm_criterios_d2,norm2_criterios_d2 * 0.01,norm2_criterios_d2])
pont_narl_d2

array([1.   , 0.002, 0.2  ])

copilando os pesos dos decisores

In [80]:
df_dec = pd.DataFrame({'d1':pont_narl,'d2':pont_narl_d2},index=["tipo de processador","Memoria em GB","Modelo de placa de video"])
df_dec

,d1,d2
tipo de processador,0.800,1.000
Memoria em GB,0.008,0.002
Modelo de placa de video,1.000,0.200


In [82]:
vetor_peso =df_dec.values
vetor_peso

array([[0.8  , 1.   ],
       [0.008, 0.002],
       [1.   , 0.2  ]])

In [190]:
# vetor resultante dos criterios
soma_pesos = np.sum(vetor_peso, axis=1)
print(soma_pesos)

[1.8  0.01 1.2 ]


### Passo 2: Avaliação das Alternativas

Avaliação do decisor 1

In [88]:
# Matriz quadrada tres por tres avalição das alternativas para o processador
av_alternativas_dc1 = {
    "ACER": pd.Series([0, 2, 2], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([-2, 0, 1], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([-2, -1, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_dc1 = pd.DataFrame(av_alternativas_dc1)

In [89]:
df_alternativas_dc1

,ACER,Dell,VAIO
ACER,0,-2,-2
Dell,2,0,-1
VAIO,2,1,0


In [90]:
vet_alter_c1 = df_alternativas_dc1.values
vet_alter_c1

array([[ 0, -2, -2],
       [ 2,  0, -1],
       [ 2,  1,  0]], dtype=int64)

In [169]:
soma_alter_c1 = np.sum(vet_alter_c1,axis=1)


In [170]:
normal_dec1 = (soma_alter_c1[0] - soma_alter_c1[0])/(soma_alter_c1[-1] - soma_alter_c1[0])


In [171]:
normal2_dec1 = (soma_alter_c1[1] - soma_alter_c1[0])/(soma_alter_c1[-1] - soma_alter_c1[0])

In [172]:
normal3_dec1 = (soma_alter_c1[2] - soma_alter_c1[0])/(soma_alter_c1[-1] - soma_alter_c1[0])

In [191]:
#vetor final processador decisor 1
processador_vetor_dc1 = np.array([normal_dec1,normal2_dec1,normal3_dec1])
print(processador_vetor_dc1)

[0.         0.71428571 1.        ]


In [102]:
# Matriz quadrada tres por tres avalição das alternativas para o memoria
av_alternativas_dc1_memoria = {
    "ACER": pd.Series([0, 0, -1], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([0, 0, 0], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([1, 0, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_memoria_dc1 = pd.DataFrame(av_alternativas_dc1_memoria)

In [103]:
df_alternativas_memoria_dc1

,ACER,Dell,VAIO
ACER,0,0,1
Dell,0,0,0
VAIO,-1,0,0


In [105]:
vet_alter_memoria_c1 = df_alternativas_memoria_dc1.values
vet_alter_memoria_c1 

array([[ 0,  0,  1],
       [ 0,  0,  0],
       [-1,  0,  0]], dtype=int64)

In [175]:
soma_alter_memoria_c1 = np.sum(vet_alter_memoria_c1,axis=1)

In [176]:
normal_memoria_dec1 = (soma_alter_memoria_c1[0] - soma_alter_memoria_c1[-1])/(soma_alter_memoria_c1[0] - soma_alter_memoria_c1[-1])


In [177]:
normal2_memoria_dec1 = (soma_alter_memoria_c1[1] - soma_alter_memoria_c1[-1])/(soma_alter_memoria_c1[0] - soma_alter_memoria_c1[-1])


In [178]:
normal3_memoria_dec1 = (soma_alter_memoria_c1[2] - soma_alter_memoria_c1[-1])/(soma_alter_memoria_c1[0] - soma_alter_memoria_c1[-1])


In [192]:
#vetor final memoria decisor 1
memoria_final_c1 = np.array([normal_memoria_dec1,normal2_memoria_dec1,normal3_memoria_dec1])
print(memoria_final_c1)

[1.  0.5 0. ]


In [113]:
# Matriz quadrada tres por tres avalição das alternativas para o placa de vdeo
av_alternativas_dc1_plcvideo = {
    "ACER": pd.Series([0, 1, -1], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([-1, 0, -3], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([1, 3, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_plvd_dc1 = pd.DataFrame(av_alternativas_dc1_plcvideo)

In [114]:
df_alternativas_plvd_dc1

,ACER,Dell,VAIO
ACER,0,-1,1
Dell,1,0,3
VAIO,-1,-3,0


In [115]:
vet_alter_pcvd_c1 = df_alternativas_plvd_dc1.values
vet_alter_pcvd_c1 

array([[ 0, -1,  1],
       [ 1,  0,  3],
       [-1, -3,  0]], dtype=int64)

In [179]:
soma_alter_plvd_c1 = np.sum(vet_alter_pcvd_c1 ,axis=1)

In [180]:
normal_plvd_dec1 = (soma_alter_plvd_c1[0] - soma_alter_plvd_c1[-1])/(soma_alter_plvd_c1[1] - soma_alter_plvd_c1[-1])

In [181]:
normal2_plvd_dec1 = (soma_alter_plvd_c1[1] - soma_alter_plvd_c1[-1])/(soma_alter_plvd_c1[1] - soma_alter_plvd_c1[-1])

In [182]:
normal3_plvd_dec1 = (soma_alter_plvd_c1[2] - soma_alter_plvd_c1[-1])/(soma_alter_plvd_c1[1] - soma_alter_plvd_c1[-1])

In [193]:
#placa de video vetor final
plv_array_dc1=np.array([normal_plvd_dec1,normal2_plvd_dec1,normal3_plvd_dec1])
print(plv_array_dc1)

[0.5 1.  0. ]


avaliação das alternativas para o decisor 2

In [130]:
# Matriz quadrada tres por tres avalição das alternativas para o processador
av_alternativas_dc2 = {
    "ACER": pd.Series([0, 1, 3], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([-1, 0, 2], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([-3, -2, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_dc2 = pd.DataFrame(av_alternativas_dc2)

In [131]:
df_alternativas_dc2

,ACER,Dell,VAIO
ACER,0,-1,-3
Dell,1,0,-2
VAIO,3,2,0


In [132]:
vet_alter_c2 = df_alternativas_dc2.values
vet_alter_c2

array([[ 0, -1, -3],
       [ 1,  0, -2],
       [ 3,  2,  0]], dtype=int64)

In [183]:
soma_alter_c2 = np.sum(vet_alter_c2,axis=1)

In [194]:
normal_plvd_dec2 = (soma_alter_c2[0] - soma_alter_c2[0])/(soma_alter_c2[-1] - soma_alter_c2[0])
normal1_plvd_dec2 = (soma_alter_c2[1] - soma_alter_c2[0])/(soma_alter_c2[-1] - soma_alter_c2[0])
normal2_plvd_dec2 = (soma_alter_c2[2] - soma_alter_c2[0])/(soma_alter_c2[-1] - soma_alter_c2[0])

processador_normal_dc2= np.array([normal_plvd_dec2,normal1_plvd_dec2,normal2_plvd_dec2])

#vetor final processador decisor 2
print(processador_normal_dc2)

[0.         0.33333333 1.        ]


In [137]:
# Matriz quadrada tres por tres avalição das alternativas para o memoria
av_alternativas_dc2_memoria = {
    "ACER": pd.Series([0, 1, 1], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([-1, 0, -1], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([-1, 1, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_memoria_dc2 = pd.DataFrame(av_alternativas_dc2_memoria)
df_alternativas_memoria_dc2

,ACER,Dell,VAIO
ACER,0,-1,-1
Dell,1,0,1
VAIO,1,-1,0


In [184]:
vet_alter_memoria_c2 = df_alternativas_memoria_dc2.values
vet_alter_memoria_c2 

soma_alter_memoria_c2 = np.sum(vet_alter_memoria_c2,axis=1)

In [195]:
normal_memoria_dec2 = (soma_alter_memoria_c2[0] - soma_alter_memoria_c2[0])/(soma_alter_memoria_c2[1] - soma_alter_memoria_c2[0])
normal2_memoria_dec2 = (soma_alter_memoria_c2[1] - soma_alter_memoria_c2[0])/(soma_alter_memoria_c2[1] - soma_alter_memoria_c2[0])
normal3_memoria_dec2 = (soma_alter_memoria_c2[2] - soma_alter_memoria_c2[0])/(soma_alter_memoria_c2[1] - soma_alter_memoria_c2[0])

memoria_nornal_c2 = np.array([normal_memoria_dec2,normal2_memoria_dec2,normal3_memoria_dec2])

#vetor final memoria decisor 2
print(memoria_nornal_c2)

[0.  1.  0.5]


In [141]:
# Matriz quadrada tres por tres avalição das alternativas para o placa de vdeo
av_alternativas_dc2_plcvideo = {
    "ACER": pd.Series([0, 2, 2], index=["ACER","Dell","VAIO"]),
    "Dell": pd.Series([-2, 0, -1], index=["ACER","Dell","VAIO"]),
    "VAIO": pd.Series([-2, 1, 0], index=["ACER","Dell","VAIO"]),
}
df_alternativas_plvd_dc2 = pd.DataFrame(av_alternativas_dc2_plcvideo)
df_alternativas_plvd_dc2

,ACER,Dell,VAIO
ACER,0,-2,-2
Dell,2,0,1
VAIO,2,-1,0


In [185]:
vet_alter_pcvd_c2 = df_alternativas_plvd_dc2.values
soma_alter_plvd_c2 = np.sum(vet_alter_pcvd_c2 ,axis=1)

In [196]:
normal_plvd_dec2 = (soma_alter_plvd_c2[0] - soma_alter_plvd_c2[0])/(soma_alter_plvd_c2[1] - soma_alter_plvd_c2[0])
normal2_plvd_dec2 = (soma_alter_plvd_c2[1] - soma_alter_plvd_c2[0])/(soma_alter_plvd_c2[1] - soma_alter_plvd_c2[0])
normal3_plvd_dec2 = (soma_alter_plvd_c2[2] - soma_alter_plvd_c2[0])/(soma_alter_plvd_c2[1] - soma_alter_plvd_c2[0])
plvd_normal_dc2 = np.array([normal_plvd_dec2,normal2_plvd_dec2,normal3_plvd_dec2])

#vetor final placa de video decisor 2
print(plvd_normal_dc2)

[0.         1.         0.71428571]


## Agregando as preferencias

processador

In [145]:
df_agr_proc = pd.DataFrame({'d1':processador_vetor_dc1,'d2':processador_normal_dc2},index=["ACER","DELL","VAIO"])
df_agr_proc

,d1,d2
ACER,0.000000,0.000000
DELL,0.714286,0.333333
VAIO,1.000000,1.000000


In [186]:
df_agr_proc_arr=df_agr_proc.values
soma_agr_proc = np.sum(df_agr_proc_arr,axis=1)

memória

In [149]:
df_agr_memoria = pd.DataFrame({'d1':memoria_final_c1,'d2':memoria_nornal_c2},index=["ACER","DELL","VAIO"])
df_agr_memoria

,d1,d2
ACER,1.0,0.0
DELL,0.5,1.0
VAIO,0.0,0.5


In [187]:
df_agr_mem_arr=df_agr_memoria.values
soma_agr_mem = np.sum(df_agr_mem_arr,axis=1)

placa de video

In [151]:
df_agr_plvd = pd.DataFrame({'d1':plv_array_dc1,'d2':plvd_normal_dc2},index=["ACER","DELL","VAIO"])
df_agr_plvd

,d1,d2
ACER,0.5,0.000000
DELL,1.0,1.000000
VAIO,0.0,0.714286


In [188]:
df_agr_plvd_arr=df_agr_plvd.values
soma_agr_plvd = np.sum(df_agr_plvd_arr,axis=1)

matriz de decisão

In [153]:
df_agr_decision = pd.DataFrame({'processador':soma_agr_proc,'memoria':soma_agr_mem,'placa de video':soma_agr_plvd},index=["ACER","DELL","VAIO"])
df_agr_decision

,processador,memoria,placa de video
ACER,0.000000,1.0,0.500000
DELL,1.047619,1.5,2.000000
VAIO,2.000000,0.5,0.714286


In [189]:
#multiplicação da matriz de decisão pelo vetor de peso(que foi calculado no processo1)
df_agr_decision_arr = df_agr_decision.values

In [157]:
resultado_final =np.dot(df_agr_decision_arr,soma_pesos)

In [158]:
result = pd.DataFrame({'pontuação final':resultado_final},index=["ACER","DELL","VAIO"])
result

,pontuação final
ACER,0.610000
DELL,4.300714
VAIO,4.462143
